Предыдущий файл - 2_dict_SMILES_text.py

#Загрузка данных

In [ ]:
import pandas as pd
import json

with open('ended_dict.json', 'r') as file:
    names_dict = json.load(file)

ds = pd.read_csv('prepared_data.csv', sep='\t', index_col = 0)
ds.head()

,ReactionSmiles,Yield
27,[CH2:1]([S:3][C:4]1[CH:26]=[CH:25][C:7]2[N:8](...,1
28,[CH2:1]([S:3][C:4]1[CH:25]=[CH:24][C:7]2[N:8](...,1
31,[H-].[Al+3].[Li+].[H-].[H-].[H-].[CH2:7]([S:9]...,1
36,[NH2:1][C:2]1[CH:3]=[CH:4][C:5]2[N:9]=[C:8]3[C...,0
41,[C:1]([NH:5][NH:6][C:7]([C:12]#[N:13])([CH:9]1...,1


#Преобразование реакций в текстовый формат

In [ ]:
def reaction_to_dict(string):

  """This function converts reaction SMILES into a dictionary of the format {'reactants': [...], 'agents': [...], 'product': [...]}"""

  dict_ = {}
  reactants = string.split('>')[0].split('.')
  dict_['reactants'] = [names_dict[i] if i in names_dict else 'пропуск' for i in reactants]
  products = string.split('>')[-1].split('.')
  dict_['product'] = [names_dict[i] if i in names_dict else 'пропуск'  for i in products]
  if string.split('>')[1] != '':
    agents = string.split('>')[1].split('.')
    dict_['agents'] = [names_dict[i] if i in names_dict else 'пропуск' for i in agents]
  return dict_

In [ ]:
def create_sentance(dict_):

  """This function creates a coherent reaction description from the IUPAC names of molecules"""

  reactants = dict_['reactants']
  if len(reactants) > 1:
    reactants = [str(i) for i in reactants]
    reactants_part = ', '.join(reactants)
    part1 = f"{reactants_part} react together"
  else:
    part1 = f'{reactants[0]} reacts'


  if 'agents' in list(dict_.keys()):
    agents = dict_['agents']
    agents = [str(i) for i in agents]
    agents_part = ', '.join(agents)
    part2 = f' in the presence of {agents_part}'
  else:
    part2 = ''
  product_part = dict_['product'][0]
  part3 = f' to produce {product_part}.'
  prompt = part1 + part2 + part3

  return prompt.capitalize()

In [ ]:
def create_text_df(df1):

    """This function creates a new dataframe with reaction descriptions instead of SMILES strings"""

    df1['dict'] = df1['ReactionSmiles'].apply(reaction_to_dict)
    df1['text'] = df1['dict'].apply(create_sentance)
    df1 = df1[['text', 'Yield']]
    return df1

In [ ]:
df = create_text_df(ds)

counter = 0
for i in ds['text']: #проверка на необработанные молекулы
    if 'пропуск' in i:
        counter += 1
print(f'Количество необработанных молекул: {counter}')

Количество необработанных молекул: 0


Преобразование таргета

In [ ]:
def reaction_class(number):
    return 'high' if number == 1 else 'low'

df['Yield'] = df['Yield'].apply(reaction_class)

Пример готового датасета

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

df = df.reset_index(drop=True)
df.head()

,text,Yield
0,"Ethyl 2-(3-ethylsulfanyl-5-hydroxy-11-methyl-6h-benzo[b][1]benzazepin-5-yl)acetate, chlorane react together in the presence of ethanol to produce ethyl 2-(3-ethylsulfanyl-11-methyl-6h-benzo[b][1]benzazepin-5-ylidene)acetate.",high
1,"Ethyl 2-(3-ethylsulfanyl-11-methyl-6h-benzo[b][1]benzazepin-5-ylidene)acetate, hydroxide, potassium(1+), chlorane react together in the presence of ethanol to produce 2-(3-ethylsulfanyl-11-methylbenzo[b][1]benzazepin-5-yl)acetic acid.",high
2,"Hydride, aluminum(3+), lithium(1+), hydride, hydride, hydride, 2-(3-ethylsulfanyl-11-methylbenzo[b][1]benzazepin-5-yl)-n,n-dimethylacetamide, ethyl acetate, hydroxide, sodium(1+) react together in the presence of oxolane to produce 2-(3-ethylsulfanyl-11-methylbenzo[b][1]benzazepin-5-yl)-n,n-dimethylethanamine.",high
3,"2,3-dihydro-1h-pyrrolo[1,2-a]benzimidazol-7-amine, acetyl acetate react together in the presence of benzene to produce n-(2,3-dihydro-1h-pyrrolo[1,2-a]benzimidazol-7-yl)acetamide.",low
4,"2-(2-tert-butylhydrazinyl)-2-cyclopropylpropanenitrile, dichloromethane, molecular bromine react together in the presence of oxidane to produce 2-(tert-butyldiazenyl)-2-cyclopropylpropanenitrile.",high


In [ ]:
df.to_csv('text_chem_data.csv', sep = '\t')

Продолжение в 4_train_test_splitting.ipynb